In [1]:
# 基于空间索引 给时序数据area打标
import findspark
findspark.init('/opt/spark')
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("dwd_to_dwm") \
    .enableHiveSupport() \
    .getOrCreate()

spark.sql("set hive.exec.dynamici.partition=true;")
spark.sql("set hive.exec.dynamic.partition.mode=nonstrict;")


DataFrame[key: string, value: string]

In [6]:
# create database time_series
spark.sql("DROP TABLE IF EXISTS dwm_time_series;")

sql = '''
CREATE TABLE IF NOT EXISTS dwm_time_series
         (id bigint,
          time TIMESTAMP,
          lon float,
          lat float)
PARTITIONED BY(area int)
STORED AS PARQUET;
'''
spark.sql(sql)

# create database job
spark.sql("DROP TABLE IF EXISTS dwm_job;")
sql = """

CREATE TABLE IF NOT EXISTS dwm_job
(id bigint,start_time TIMESTAMP,end_time TIMESTAMP)
PARTITIONED BY(yearMonth string)
STORED AS PARQUET;

"""
spark.sql(sql)

# 创建输出表
#CREATE TABLE output(id SERIAL PRIMARY KEY,job_id int4);

print("初始化 dwm_time_series，dwm_job成功")

初始化 dwm_time_series，dwm_job成功


In [10]:
# 加载job 表=> 

sql = '''
INSERT INTO dwm_job partition(yearMonth)
SELECT
 id as id,
 start_time as start_time,
 end_time as end_time,
 yearMonth as yearMonth
 FROM dwd_job;
'''
result = spark.sql(sql)

# 查看一条数据
result = spark.sql("SELECT * FROM dwd_job LIMIT 1")
result.show()



INSERT INTO dwm_job partition(yearMonth)
SELECT
 id as id,
 start_time as start_time,
 end_time as end_time,
 yearMonth as yearMonth
 FROM dwd_job;

+-----+-------------------+-------------------+---------+
|   id|         start_time|           end_time|yearMonth|
+-----+-------------------+-------------------+---------+
|10208|2025-03-01 00:00:00|2025-03-01 01:00:00|   202503|
+-----+-------------------+-------------------+---------+



In [ ]:
# 利用空间索引给数据打标